In [ ]:
import os
import numpy as np
from skimage import io
from random import shuffle
from scipy import misc
from tqdm import tqdm
import tflearn
from tflearn.layers.conv import conv_3d, max_pool_3d, conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
import tensorflow as tf

In [ ]:
def get_testing_data_2d():
    
    EOSINOPHIL = os.listdir("TEST/EOSINOPHIL/")
    LYMPHOCYTE = os.listdir("TEST/LYMPHOCYTE/")
    MONOCYTE   = os.listdir("TEST/MONOCYTE/")
    NEUTROPHIL = os.listdir("TEST/NEUTROPHIL/")
    

    testing_data = []
    

    for (imgE, imgL, imgM, imgN) in tqdm(zip(EOSINOPHIL, LYMPHOCYTE, MONOCYTE, NEUTROPHIL)):

        img_e = io.imread('TEST/EOSINOPHIL/' + imgE , as_grey=True)
        img_l = io.imread('TEST/LYMPHOCYTE/' + imgL , as_grey=True)
        img_m = io.imread('TEST/MONOCYTE/'   + imgM , as_grey=True)
        img_n = io.imread('TEST/NEUTROPHIL/' + imgN , as_grey=True)
        
        img_e = misc.imresize(img_e , (240, 240))
        img_l = misc.imresize(img_l , (240, 240))
        img_m = misc.imresize(img_m , (240, 240))
        img_n = misc.imresize(img_n , (240, 240))
        
        testing_data.append([np.array(img_e), np.array([1, 0, 0, 0])])
        testing_data.append([np.array(img_l), np.array([0, 1, 0, 0])])
        testing_data.append([np.array(img_m), np.array([0, 0, 1, 0])])
        testing_data.append([np.array(img_n), np.array([0, 0, 0, 1])])
    
    shuffle(testing_data)
    
    return testing_data


In [ ]:
def get_training_data_2d():
    
    EOSINOPHIL = os.listdir("TRAIN/EOSINOPHIL/")
    LYMPHOCYTE = os.listdir("TRAIN/LYMPHOCYTE/")
    MONOCYTE   = os.listdir("TRAIN/MONOCYTE/")
    NEUTROPHIL = os.listdir("TRAIN/NEUTROPHIL/")
    

    training_data = []
    

    for (imgE, imgL, imgM, imgN) in tqdm(zip(EOSINOPHIL, LYMPHOCYTE, MONOCYTE, NEUTROPHIL)):

        img_e = io.imread('TRAIN/EOSINOPHIL/' + imgE , as_grey=True)
        img_l = io.imread('TRAIN/LYMPHOCYTE/' + imgL , as_grey=True)
        img_m = io.imread('TRAIN/MONOCYTE/'   + imgM , as_grey=True)
        img_n = io.imread('TRAIN/NEUTROPHIL/' + imgN , as_grey=True)
        
        img_e = misc.imresize(img_e , (240, 240))
        img_l = misc.imresize(img_l , (240, 240))
        img_m = misc.imresize(img_m , (240, 240))
        img_n = misc.imresize(img_n , (240, 240))
        
        training_data.append([np.array(img_e), np.array([1, 0, 0, 0])])
        training_data.append([np.array(img_l), np.array([0, 1, 0, 0])])
        training_data.append([np.array(img_m), np.array([0, 0, 1, 0])])
        training_data.append([np.array(img_n), np.array([0, 0, 0, 1])])
        
    
    shuffle(training_data)
    
    return training_data


In [ ]:
training_data = get_training_data_2d()
testing_data  = get_testing_data_2d()

In [ ]:
x_train = np.array([i[0] for i in training_data]).reshape(-1,240,240,1)
y_train = [i[1] for i in training_data]

x_test = np.array([i[0] for i in testing_data[:3000]]).reshape(-1,240,240,1)
y_test = [i[1] for i in testing_data[:3000]]

In [ ]:
convnet = input_data(shape=[None, 240, 240, 1], name='input')

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 128, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)


convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 4, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=1e-3, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet)

In [ ]:
model.fit({'input': x_train}, {'targets': y_train}, n_epoch=5, validation_set=({'input': x_test}, {'targets': y_test}), 
    snapshot_step=500, show_metric=True)

In [ ]:
# Save the model
model.save("model.tfl")
print(model)